In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 7)
print("success")

success


In [2]:
model.load_state_dict(torch.load('library_model.pth'))
print("success")

success


In [3]:
device = torch.device('cuda')
model = model.to(device)
print("success")

success


In [4]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)
print("success")

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

success


In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

print("success")

success


In [6]:
from jetbot import Robot

robot = Robot()
print("success")

success


In [7]:
import torch.nn.functional as F
import time
target_class = 4 ##找的類別
##取得list裡最大值的索引位置(藉此判斷類別)
def argmax(lst):
    return max(range(len(lst)), key=lst.__getitem__)
##
recent_class=[]
recent_class_count=[0,0,0,0,0,0,0]
def update(change):
    global robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    yf =y.flatten()
    ##偵測可能之類別
    class_acc = [] 
    for i in range(len(yf)):
        class_acc.append(float(yf[i]))
    detect_class = argmax(class_acc)
    
    ############Beta
    
    #添加到最近偵測之類別的陣列
    if len(recent_class)>100:
        recent_class.pop(0)
    recent_class.append(detect_class)
    ##
    
    ##找出近100次的類別大概都是哪一個
    for i in range(len(recent_class_count)):
        recent_class_count[i] = recent_class.count(i)
    offten = argmax(recent_class_count)
    ##
    
#     ##################
    
    acc = class_acc[detect_class]
    
    print(detect_class)##偵測的類別
    print("acc:"+str(acc))##命中率
    print("recent"+str(offten))
    ##根據偵測類別進行之行為
    if detect_class == target_class:
        robot.set_motors(0.6,0.65)
        time.sleep(1)
    else:##不是尋求的類別
        robot.left(0.5)
    ##
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize
print("success")

5
acc:0.949223518371582
recent5
success


In [8]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

5
acc:0.9674591422080994
recent5
0
acc:0.5226999521255493
recent5
4
acc:0.8170533776283264
recent5
4
acc:0.6202071309089661
recent4
4
acc:0.6547799110412598
recent4
6
acc:0.37473535537719727
recent4
2
acc:0.34610822796821594
recent4
0
acc:0.7529361844062805
recent4
6
acc:0.743560791015625
recent4
6
acc:0.6067875027656555
recent4
4
acc:0.2716955542564392
recent4
0
acc:0.7508214712142944
recent4
5
acc:0.9857014417648315
recent4
5
acc:0.9445356130599976
recent4
5
acc:0.708193838596344
recent5
3
acc:0.4466232359409332
recent5
6
acc:0.39953380823135376
recent5
3
acc:0.5184438228607178
recent5
6
acc:0.46708664298057556
recent5
5
acc:0.29081958532333374
recent5
3
acc:0.38521528244018555
recent5
1
acc:0.2670764625072479
recent5
6
acc:0.28203099966049194
recent5
5
acc:0.30357763171195984
recent5
3
acc:0.3114203214645386
recent5
3
acc:0.3746829926967621
recent5
3
acc:0.3157031238079071
recent5
6
acc:0.27270790934562683
recent5
6
acc:0.3898150622844696
recent6
0
acc:0.3114730715751648
recent6
1
a

5
acc:0.23442359268665314
recent1
1
acc:0.3507375419139862
recent1
5
acc:0.7815297245979309
recent1
1
acc:0.49447086453437805
recent1
1
acc:0.4785747230052948
recent1
5
acc:0.6127392649650574
recent1
5
acc:0.5805501341819763
recent1
5
acc:0.7618107199668884
recent1
5
acc:0.8613425493240356
recent1
5
acc:0.9885856509208679
recent1
5
acc:0.8059489727020264
recent1
5
acc:0.9634605050086975
recent1
5
acc:0.6242590546607971
recent1
5
acc:0.9140153527259827
recent1
5
acc:0.5126345753669739
recent1
5
acc:0.7314561009407043
recent1
5
acc:0.5839244723320007
recent1
5
acc:0.9487848281860352
recent1
5
acc:0.7012529969215393
recent1
5
acc:0.7662087082862854
recent1
5
acc:0.7563496828079224
recent1
5
acc:0.8203742504119873
recent1
5
acc:0.7442765235900879
recent1
5
acc:0.6690165400505066
recent1
5
acc:0.9658200740814209
recent1
5
acc:0.9574836492538452
recent1
5
acc:0.9718536138534546
recent1
5
acc:0.9617391228675842
recent1
5
acc:0.942669153213501
recent1
5
acc:0.9366246461868286
recent1
5
acc:0.9

1
acc:0.2601304054260254
recent5
1
acc:0.36688467860221863
recent5
4
acc:0.25356683135032654
recent5
4
acc:0.2518422603607178
recent5
0
acc:0.40994733572006226
recent5
1
acc:0.47880253195762634
recent5
1
acc:0.3932340145111084
recent5
1
acc:0.35136574506759644
recent1
1
acc:0.3165211081504822
recent1
4
acc:0.2802387773990631
recent1
1
acc:0.45643094182014465
recent1
0
acc:0.3208683729171753
recent1
1
acc:0.34603506326675415
recent1
1
acc:0.29969361424446106
recent1
4
acc:0.2997710704803467
recent1
1
acc:0.34740835428237915
recent1
1
acc:0.3655838072299957
recent1
1
acc:0.6179559826850891
recent1
1
acc:0.2600107491016388
recent1
1
acc:0.25465285778045654
recent1
1
acc:0.35614943504333496
recent1
2
acc:0.33535823225975037
recent1
1
acc:0.3972414433956146
recent1
2
acc:0.3009590208530426
recent1
2
acc:0.35412925481796265
recent1
4
acc:0.2468157857656479
recent1
1
acc:0.5217717885971069
recent1
2
acc:0.39599522948265076
recent1
4
acc:0.31264936923980713
recent1
1
acc:0.5498725175857544
rec

In [11]:
camera.unobserve(update, names='value')
robot.stop()

0
acc:0.5418421030044556
recent0
0
acc:0.8782258629798889
recent0


In [12]:
robot.stop()

In [10]:
target_class=6

0
acc:0.7885289788246155
recent0
0
acc:0.8445595502853394
recent0
0
acc:0.8393306732177734
recent0
0
acc:0.7211419343948364
recent0
0
acc:0.9188538193702698
recent0
0
acc:0.9259771108627319
recent0
0
acc:0.8952715992927551
recent0
0
acc:0.8817713260650635
recent0
0
acc:0.7955177426338196
recent0
0
acc:0.7185388803482056
recent0
0
acc:0.9154590964317322
recent0
0
acc:0.8550878167152405
recent0
0
acc:0.8471789956092834
recent0
0
acc:0.5479341149330139
recent0
0
acc:0.4249364733695984
recent0
0
acc:0.5159709453582764
recent0
0
acc:0.5529305934906006
recent0
0
acc:0.8069785833358765
recent0
0
acc:0.7217302918434143
recent0
0
acc:0.9064545631408691
recent0
0
acc:0.8930279612541199
recent0
0
acc:0.8001454472541809
recent0
0
acc:0.4924025237560272
recent0
0
acc:0.45336881279945374
recent0
0
acc:0.6608314514160156
recent0
0
acc:0.9560035467147827
recent0
0
acc:0.953734278678894
recent0
0
acc:0.8469412922859192
recent0
6
acc:0.5145735144615173
recent0
5
acc:0.9574344754219055
recent0
5
acc:0.99

0
acc:0.6054697036743164
recent0
2
acc:0.4646972715854645
recent0
0
acc:0.6553825736045837
recent0
0
acc:0.6491380929946899
recent0
0
acc:0.4800925850868225
recent0
6
acc:0.44265472888946533
recent0
5
acc:0.976892352104187
recent0
5
acc:0.9688566327095032
recent0
5
acc:0.9660727977752686
recent0
5
acc:0.7526565790176392
recent0
5
acc:0.5789936184883118
recent0
5
acc:0.5013806819915771
recent0
5
acc:0.4908379912376404
recent0
5
acc:0.6142356991767883
recent0
5
acc:0.2802775502204895
recent0
6
acc:0.2484905868768692
recent0
3
acc:0.5376591086387634
recent0
5
acc:0.6894538998603821
recent0
5
acc:0.8088842630386353
recent0
5
acc:0.8241057395935059
recent0
2
acc:0.26220059394836426
recent0
3
acc:0.2998805642127991
recent0
4
acc:0.30372366309165955
recent0
1
acc:0.4767133295536041
recent0
1
acc:0.5584427118301392
recent0
1
acc:0.8546109199523926
recent0
1
acc:0.8652427196502686
recent0
1
acc:0.9931418299674988
recent0
1
acc:0.955585241317749
recent0
1
acc:0.9761649370193481
recent0
1
acc:0.8

1
acc:0.6957128047943115
recent0
1
acc:0.9108661413192749
recent0
1
acc:0.8995716571807861
recent0
1
acc:0.9539924263954163
recent0
1
acc:0.9377379417419434
recent0
1
acc:0.7112230062484741
recent0
1
acc:0.664497971534729
recent0
1
acc:0.5129146575927734
recent0
0
acc:0.7910468578338623
recent0
0
acc:0.7630133032798767
recent0
0
acc:0.8857501745223999
recent0
0
acc:0.6785195469856262
recent0
0
acc:0.919955313205719
recent0
0
acc:0.6809632778167725
recent0
0
acc:0.5364672541618347
recent0
0
acc:0.59331214427948
recent0
0
acc:0.5437966585159302
recent0
0
acc:0.530906617641449
recent0
0
acc:0.542091965675354
recent0
6
acc:0.45733246207237244
recent0
4
acc:0.9895123839378357
recent0
4
acc:0.6990218162536621
recent0
4
acc:0.8824208974838257
recent0
0
acc:0.7617809176445007
recent0
0
acc:0.7160105109214783
recent0
0
acc:0.5216281414031982
recent0
6
acc:0.47224196791648865
recent0
5
acc:0.8172917366027832
recent0
5
acc:0.757584273815155
recent0
5
acc:0.6973463892936707
recent0
6
acc:0.2398177

0
acc:0.6273715496063232
recent0
0
acc:0.43606022000312805
recent0
3
acc:0.4115954339504242
recent0
6
acc:0.38588768243789673
recent0
1
acc:0.46954357624053955
recent0
5
acc:0.5166752934455872
recent0
5
acc:0.5083677768707275
recent0
1
acc:0.3736218214035034
recent0
1
acc:0.7027904987335205
recent0
1
acc:0.9511411190032959
recent0
1
acc:0.7860686182975769
recent0
0
acc:0.38678279519081116
recent0
1
acc:0.4576739966869354
recent0
0
acc:0.6094725131988525
recent0
0
acc:0.5035357475280762
recent0
0
acc:0.4301118552684784
recent0
6
acc:0.534562349319458
recent0
4
acc:0.6931142210960388
recent0
4
acc:0.9254146218299866
recent0
3
acc:0.5539926290512085
recent0
5
acc:0.3728169798851013
recent0
3
acc:0.2754195034503937
recent0
3
acc:0.48903173208236694
recent0
6
acc:0.608109712600708
recent0
5
acc:0.5751520991325378
recent0
5
acc:0.8231533169746399
recent0
3
acc:0.515146017074585
recent0
3
acc:0.5690577030181885
recent0
3
acc:0.4494682848453522
recent0
6
acc:0.5316100716590881
recent0
0
acc:0.

2
acc:0.6672529578208923
recent0
6
acc:0.5056997537612915
recent0
1
acc:0.5508215427398682
recent0
1
acc:0.3452654182910919
recent0
6
acc:0.7382892370223999
recent0
1
acc:0.39972952008247375
recent0
6
acc:0.40416789054870605
recent0
1
acc:0.3528759479522705
recent0
6
acc:0.42282989621162415
recent0
6
acc:0.3791101276874542
recent0
0
acc:0.4032565951347351
recent0
1
acc:0.3462892174720764
recent0
0
acc:0.6087185144424438
recent0
1
acc:0.5934159755706787
recent0
0
acc:0.4342103898525238
recent0
1
acc:0.4528205394744873
recent0
1
acc:0.39361026883125305
recent0
6
acc:0.3836539089679718
recent0
0
acc:0.5453681349754333
recent0
0
acc:0.480991393327713
recent0
6
acc:0.3115398585796356
recent0
6
acc:0.608243465423584
recent0
1
acc:0.36781322956085205
recent0
1
acc:0.5345920324325562
recent0
1
acc:0.38780805468559265
recent0
1
acc:0.3519495725631714
recent0
1
acc:0.337237685918808
recent0
1
acc:0.366321325302124
recent0
1
acc:0.318875253200531
recent0
6
acc:0.5926915407180786
recent0
0
acc:0.4

1
acc:0.47811174392700195
recent5
1
acc:0.3608434498310089
recent5
1
acc:0.7154899835586548
recent5
1
acc:0.6688011288642883
recent5
1
acc:0.5704330801963806
recent5
1
acc:0.453811913728714
recent5
0
acc:0.2467559576034546
recent5
1
acc:0.24437491595745087
recent5
2
acc:0.33984944224357605
recent5
4
acc:0.312930703163147
recent5
4
acc:0.5082327127456665
recent5
1
acc:0.2278020679950714
recent5
4
acc:0.3257099688053131
recent5
4
acc:0.3865521252155304
recent5
6
acc:0.29619377851486206
recent5
5
acc:0.7556833624839783
recent5
5
acc:0.9469881057739258
recent5
5
acc:0.3982726037502289
recent5
5
acc:0.9193899035453796
recent5
5
acc:0.9533682465553284
recent5
5
acc:0.9093465209007263
recent5
5
acc:0.8108742237091064
recent5
5
acc:0.9252961277961731
recent5
5
acc:0.9418017864227295
recent5
5
acc:0.5230429172515869
recent5
5
acc:0.8521342277526855
recent5
5
acc:0.9815030097961426
recent5
5
acc:0.9357934594154358
recent5
5
acc:0.8707091808319092
recent5
5
acc:0.6851035952568054
recent5
5
acc:0.

0
acc:0.6454627513885498
recent4
0
acc:0.8993257880210876
recent0
0
acc:0.9654030203819275
recent0
0
acc:0.5265790224075317
recent0
6
acc:0.4897395670413971
recent0
6
acc:0.5256413817405701
recent0
0
acc:0.8093997836112976
recent0
0
acc:0.5952366590499878
recent0
0
acc:0.5204406976699829
recent0
0
acc:0.772031307220459
recent0
6
acc:0.5650346875190735
recent0
0
acc:0.7275484800338745
recent0
6
acc:0.5990113019943237
recent0
0
acc:0.5358184576034546
recent0
0
acc:0.441116064786911
recent0
0
acc:0.8319979906082153
recent0
0
acc:0.8650633096694946
recent0


In [ ]:
##無人工廠0 ##直昇機1 ##平衡機2 ##人臉辨識3 ##復健機4 ##健康檢查5 ##TV 6